In [4]:
!git clone https://github.com/IlyaSk10/POS_Tagger_with_using_CNN/
%cd POS_Tagger_with_using_CNN
!unzip SYNTAGRUS_texts.zip
!head syntagrus_full.ud
from data_preparation.converter_from_ud_to_txt import UDConverter
UDConverter.convert_from_conllu("syntagrus_full.ud", "syntagrus_fixed.txt")
!head syntagrus_fixed.txt
text_data= "syntagrus_fixed.txt"

fatal: destination path 'POS_Tagger_with_using_CNN' already exists and is not an empty directory.
/content/POS_Tagger_with_using_CNN
Archive:  SYNTAGRUS_texts.zip
replace syntagrus_full.ud? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: syntagrus_full.ud       
1	Начальник	начальник	NOUN	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
2	областного	областной	ADJ	Case=Gen|Degree=Pos|Gender=Neut|Number=Sing
3	управления	управление	NOUN	Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing
4	связи	связь	NOUN	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing
5	Семен	семен	NOUN	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
6	Еремеевич	еремеевич	NOUN	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
7	был	быть	VERB	Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act
8	человек	человек	NOUN	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
9	простой	простой	ADJ	Case=Nom|Degree=Pos|Gender=Masc|Number=Sing
10	,	,	PUNCT	_
Начальник	начальник	NOUN	Case=Nom|Gender=Masc|Number=Sing
областного	областной

In [0]:
def load_from_file(file):
  with open(file,'r',encoding='utf-8') as f:
    read_file=f.read().split('\n')
  return read_file

In [0]:
dataset=load_from_file(text_data)

In [7]:
import string
dataset = list(filter(None, dataset))
text = ''.join([('' if iteration.split('\t')[2]=='PUNCT' else ' ')+iteration.split('\t')[0] for iteration in dataset[:200]]).strip()
print(text)

Начальник областного управления связи Семен Еремеевич был человек простой, приходил на работу всегда вовремя, здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом" Муха". В приемной его с утра ожидали посетители,- кое-кто с важными делами, а кое-кто и с такими, которые легко можно было решить в нижестоящих инстанциях, не затрудняя Семена Еремеевича. Однако стиль работы Семена Еремеевича заключался в том, чтобы принимать всех желающих и лично вникать в дело. Приемная была обставлена просто, но по-деловому. У двери стоял стол секретарши, на столе- пишущая машинка с широкой кареткой. В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того, чтобы заглушать голос начальника, доносившийся из кабинета, так_как, бесспорно, среди посетителей могли находиться и случайные люди. Кабинет отличался скромностью, присущей Семену Еремеевичу. В глубине стоял широкий письменный стол с бронзовыми чернильницами и перед ним два кожаных кресла. Сп

In [0]:
def clear_text (dataset):
  clean_dataset=[]
  for sentence in dataset:
    if sentence.split('\t')[0]!='.' and sentence.split('\t')[2]!='PUNCT':
      clean_dataset.append([sentence.split('\t')[0].lower(),sentence.split('\t')[2]])
    elif sentence.split('\t')[0]=='.':
      clean_dataset.append([sentence.split('\t')[0].lower(),sentence.split('\t')[2]])
    else:
      continue
  return clean_dataset

In [0]:
clean_dataset=clear_text(dataset)

In [10]:
print(clean_dataset[:10])

[['начальник', 'NOUN'], ['областного', 'ADJ'], ['управления', 'NOUN'], ['связи', 'NOUN'], ['семен', 'NOUN'], ['еремеевич', 'NOUN'], ['был', 'VERB'], ['человек', 'NOUN'], ['простой', 'ADJ'], ['приходил', 'VERB']]


In [0]:
def concatenate_words(dataset):
  concat_words=[]
  concat_labels=[]
  sentence=[]
  labels=[]
  for word in dataset:
    if word[1]!='PUNCT':
      sentence.append(word[0])
      labels.append(word[1])
    else:
      concat_words.append([word for word in sentence])
      concat_labels.append([label for label in labels])
      sentence.clear()
      labels.clear()
  return concat_words,concat_labels

In [0]:
text,labels_sen=concatenate_words(clean_dataset)

In [13]:
clean_dataset=[line for line in clean_dataset if line!='']
MAX_LEN_TOKEN=max(len(line[0]) for line in clean_dataset)
NUMBER_UNIQUE_TOKEN=len(set(line[0] for line in clean_dataset if line[1]!='PUNCT'))
MAX_LEN_SENTENCE=max(len(sentence) for sentence in text)
print('Максимальная длина токена', MAX_LEN_TOKEN)
print('Количество уникальных токенов' , NUMBER_UNIQUE_TOKEN)
print('Максимальная длина предложения' , MAX_LEN_SENTENCE)

Максимальная длина токена 31
Количество уникальных токенов 104251
Максимальная длина предложения 216


In [14]:
labels = ['<NOTAG>'] + sorted({token[1] for token in clean_dataset})
label2id = {label: i+1 for i, label in enumerate(labels)}
print('Метки частей речи' , label2id)

Метки частей речи {'<NOTAG>': 1, 'ADJ': 2, 'ADP': 3, 'ADV': 4, 'CONJ': 5, 'DET': 6, 'H': 7, 'INTJ': 8, 'NOUN': 9, 'NUM': 10, 'PART': 11, 'PRON': 12, 'PUNCT': 13, 'VERB': 14}


In [0]:
from collections import Counter
def build_vocab(text,pad_symbol='<PAD>'):
  sentence=' '.join(line[0].lower() for line in text)
  mydict = dict((j, i+1) for i, j in enumerate(set(sentence)))
  mydict.update({pad_symbol:max(mydict.values())+1})
  return mydict

def counter(dataset,most_freq_symbols):
  if most_freq_symbols is None:
    most_freq_symbols=5
  sentence=' '.join([word[0] for word in dataset])
  return Counter(list(sentence)).most_common(most_freq_symbols)    


In [16]:
print('Символы словаря',build_vocab(text=clean_dataset[:10]),'\n','Наиболее частотные символы',counter(clean_dataset[:10],most_freq_symbols=10))

Символы словаря {'н': 1, 'в': 2, 'к': 3, 'с': 4, 'д': 5, 'л': 6, 'п': 7, 'ч': 8, 'а': 9, 'з': 10, 'б': 11, 'т': 12, 'р': 13, 'м': 14, 'г': 15, 'и': 16, ' ': 17, 'я': 18, 'у': 19, 'ы': 20, 'й': 21, 'х': 22, 'о': 23, 'е': 24, 'ь': 25, '<PAD>': 26} 
 Наиболее частотные символы [(' ', 9), ('е', 9), ('о', 7), ('л', 6), ('и', 6), ('н', 5), ('а', 4), ('с', 4), ('р', 4), ('в', 4)]


In [0]:
vocab=build_vocab(text=clean_dataset)

import numpy as np

A=np.zeros(shape=(len(text),MAX_LEN_SENTENCE,MAX_LEN_TOKEN))
B=np.zeros(shape=(len(text),MAX_LEN_SENTENCE,1))
for i in range(len(text)):
  for j,word in enumerate(text[i]):
    conv=[number for letter in word for symbol,number in vocab.items() if letter==symbol]
    conv1=[number for symbol,number in label2id.items() if labels_sen[i][j]==symbol]
    #print(word,j,conv)
    np.put(A[i,j],[k for k in range(len(word))],conv)
    np.put(B[i,j],0,conv1)


In [0]:
def add_pad(words2id,text,pad_symbol,max_len_token,add_boundary=True):
  mat_with_boundary=np.zeros(shape=(len(text),MAX_LEN_SENTENCE,MAX_LEN_TOKEN+1),dtype=int)
  for i in range(len(text)):
    for j in range(len(text[i])):
      if add_boundary==True:
        mat_with_boundary[i,j]=np.insert(words2id[i,j],0,pad_symbol)
      else:
        continue
  return words2id if (add_boundary==False) else mat_with_boundary

In [0]:
mat_with_pad=add_pad(A,text,pad_symbol=0,max_len_token=MAX_LEN_TOKEN,add_boundary=True)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(words, label_to_id, test_size=0.33, random_state=42)

from keras.utils import to_categorical

y_train_cat=to_categorical(y_train, num_classes=len(labels)+1)
y_test_cat=to_categorical(y_test, num_classes=len(labels)+1)

Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation,Embedding,Bidirectional,LSTM

# создание сети
model = Sequential()
vocab_size=NUMBER_UNIQUE_TOKEN
num_classes=len(labels)+1
model.add(Embedding(vocab_size, output_dim= MAX_LEN_TOKEN, input_length=MAX_LEN_TOKEN))
model.add(Bidirectional(LSTM(units=32, input_shape=(NUMBER_UNIQUE_TOKEN, MAX_LEN_TOKEN),dropout = 0.2)))
model.add(Dense(units=num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# обучение модели
model.fit(X_train, y_train_cat, batch_size = 64,epochs=1,validation_data = (X_test, y_test_cat),verbose = 1)